# Lab10

---

## Task

Реализовать решение уравнения теплопроводности по двум схемам: одной из неявных и явной.

Посмотреть на поведение решения по явной схеме при несоблюдении условий устойчивости.
Результаты выводить либо графически (поверхность), либо численно (матрицу значений).

# Solution

---

In [ ]:
import math
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.cm as cm

### Явная схема

In [ ]:
def grid(segment, n):
    a, b = segment
    return np.linspace(a, b, n)

def is_sustained(kappa, tau, h):
    if kappa * tau * 2 > h * h:
        return True
    return False

def boundary_constraints(u_, a, Nx, Nt, t):
    mesh_x = grid( (0, a), Nx + 1)
    mesh_t = grid( (0, t), Nt + 1)
    u = np.zeros( (Nx + 1, Nt + 1) )

    for i in range(Nx + 1):
        u[i, 0] = u_(mesh_x[i], 0)

    for i in range(Nx + 1):
        u[0, i] = u_(mesh_x[0], mesh_t[i])
        u[Nx, i] = u_(mesh_x[Nx], mesh_t[i])

    return mesh_x, mesh_t, u


def explicit_scheme(u, f, a, t, kappa, Nx, Nt):
    mesh_x, mesh_t, u_= boundary_constraints(u, a, Nx, Nt, t)

    h = t / Nx
    tau =  t / Nt

    for t in range(1, Nt + 1):
        for x in range(1, Nx):
            diff = u_[x - 1, t - 1] - 2 * u_[x, t - 1] + u_[x + 1, t - 1]
            u_[x, t] = u_[x, t - 1] + tau * (kappa / h**2 * diff + f(mesh_x[x], mesh_t[t - 1]))

    sustained = is_sustained(kappa, tau, h)
    X, T = np.meshgrid(mesh_x, mesh_t)

    return X, T, u_, sustained

### Неявная схема

In [ ]:
def implicit_scheme(u, f, a, t, kappa, Nx, Nt):
    mesh_x, mesh_t, u_ = boundary_constraints(u, a, Nx, Nt, t)
    
    h = t / Nx
    tau =  t / Nt

    for t in range(1, Nt + 1):
        lbase = np.zeros( (Nx + 1, Nx + 1) )
        rbase = np.zeros(Nx + 1)

        lbase[0, 0] = -(tau * kappa / h + 1)
        lbase[0, 1] = tau * kappa / h
        rbase[0] = -u_[0, t - 1] - tau * f(mesh_x[0], mesh_t[t])

        lbase[Nx, Nx] = tau * kappa / h - 1
        lbase[Nx, Nx - 1] = -tau * kappa / h  
        rbase[Nx] = -u_[Nx, t - 1] - tau * f(mesh_x[Nx], mesh_t[t])

        for x in range(1, Nx):
            lbase[x, x] = -2 * tau * kappa / (h * h) - 1
            lbase[x, x - 1] = lbase[x, x + 1] = tau * kappa / (h * h)

            rbase[x] = -u_[x, t - 1] - tau * f(mesh_x[x], mesh_t[t])

        u_[:, t] = np.linalg.solve(lbase, rbase)

    X, T = np.meshgrid(mesh_x, mesh_t)

    return X, T, u_

## Experimental reseach

In [ ]:
def experiment(u, f, a, t, kappa, Nx, Nt):
    fig = plt.figure(figsize=(25, 10))
    
    ax_left = fig.add_subplot(121, projection = "3d")
    ax_right = fig.add_subplot(122, projection = "3d")

    mesh_x, mesh_t, result, is_sustained = explicit_scheme(u, f, a, t, kappa, Nx, Nt)
    ax_left.plot_surface(mesh_x, mesh_t, result, cmap=cm.viridis)

    mesh_x, mesh_t, result               = implicit_scheme(u, f, a, t, kappa, Nx, Nt)
    ax_right.plot_surface(mesh_x, mesh_t, result, cmap=cm.viridis)

    ax_left.set_title(f"Explicit. Is_sustained: {is_sustained}")
    ax_right.set_title("Implicit")
    
    plt.show()

In [ ]:
kappa = 0.01
a, T = 10, 10
Nx, Nt = 20, 20

u = lambda x, t: x * t
f = lambda x, t: x

experiment(u, f, a, T, kappa, Nx, Nt)

In [ ]:
kappa = 0.01
a, T = 10, 10
Nx, Nt = 20, 20

u = lambda x, t: x**2 * t
f = lambda x, t: x**2 - 2 * t

experiment(u, f, a, T, kappa, Nx, Nt)

In [ ]:
kappa = 0.1
a, T = 10, 10
Nx, Nt = 100, 100

u = lambda x, t: x**2 * t
f = lambda x, t: 2 * kappa - 2 * t

experiment(u, f, a, T, kappa, Nx, Nt)

In [ ]:
kappa = 0.01
a, T = 10, 10
Nx, Nt = 20, 20

u = lambda x, t: x ** 2 / 2 - t ** 2 / 2
f = lambda x, t: 2 * kappa  - 2 * t / 2

experiment(u, f, a, T, kappa, Nx, Nt)

In [ ]:
kappa = 0.001
a, T = 10, 10
Nx, Nt = 20, 20

u = lambda x, t: x**3 + t**3
f = lambda x, t: 3 * t**2 - kappa * 6 * x

experiment(u, f, a, T, kappa, Nx, Nt)